<a href="https://colab.research.google.com/github/lohan20s/KNeighbors_Book_Recommendation/blob/main/fcc_book_recommendation_knn_SL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
#load books
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

#remove duplicate rows (same book title but different isbns)
df_books.drop_duplicates(subset=['title'],inplace=True)
print(df_books.head())

#load ratings
df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})
print(df_ratings.head())


In [ ]:
#Graph the dataset, you will notice that most books are not rated frequently (rating = 0)
plt.figure(figsize=(8, 5))
plt.hist(df_ratings["rating"], bins=11, edgecolor="black", alpha=0.7)

plt.xlabel("Ratings")
plt.ylabel("Frequency")
plt.title("Distribution of Book Ratings")
plt.show()


In [ ]:
#remove outliers:remove rows where ratings are zero as that means the book was not rated by the user
#df_ratings=df_ratings[df_ratings["rating"]>0]

#find users with less than 200 ratings and books with less than 100 ratings and remove them from the dataset
user_counts=df_ratings["user"].value_counts()
user_keep=user_counts[user_counts>=200].index
book_counts=df_ratings["isbn"].value_counts()
book_keep=book_counts[book_counts>=100].index
df_ratings_filtered=df_ratings[df_ratings["user"].isin(user_keep)& df_ratings["isbn"].isin(book_keep)].reset_index(drop=True)


In [ ]:
#merge df_ratings with df_books
df_merged = pd.merge(df_books,df_ratings_filtered,left_on = 'isbn', right_on = 'isbn', how = 'right')

#aggregate ratings values across users by making a pivot table and converting to csr matrix
df_pivottable=df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)
df_ratings_matrix = csr_matrix(df_pivottable.values)


In [ ]:
#use NearestNeighbors to develop a model that shows books that are similar to a given book.
model= NearestNeighbors(metric='cosine',n_neighbors=6)
model.fit(df_ratings_matrix)

In [ ]:
#Create a function named get_recommends that takes a book title (from the dataset) as an argument
#and returns a list of 5 similar books with their distances from the book argument.
def get_recommends(book = ""):
  book_rating=csr_matrix(df_pivottable.loc[book])
  distances, indices = model.kneighbors(book_rating)
  indices=indices.flatten( )
  distances=distances.flatten( )
  booklist=[]
  for i in range(len(indices)-1,1,-1):
    booklist.append([df_pivottable.iloc[indices[i]].name,distances[i]])
  final_book_list=[book,booklist]
  return(final_book_list)

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()